In [1]:
!sudo apt update
!sudo apt install libboost-dev libboost-program-options-dev libboost-system-dev libboost-thread-dev libboost-math-dev libboost-test-dev libboost-python-dev zlib1g-dev cmake python3 python3-pip
!pip3 install vowpalwabbit

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:14 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [8

In [0]:
from __future__ import print_function
from vowpalwabbit import pyvw

This is an implementation for a POS tagger.

The main functions used in this are similiar to the C++ code for the dependency parser.

The POS tagger has only 5 tags which it has to predict.

The 5th one being a dummy tag.





In [0]:
DET  = 1
NOUN = 2
VERB = 3
ADJ  = 4


DUMMY  = 5      
# Dummy tag


# the dataset

my_dataset = [ [(DET , 'the'),
                (NOUN, 'monster'),
                (VERB, 'ate'),
                (DET , 'a'),
                (ADJ , 'big'),
                (NOUN, 'sandwich')],
               [(DET , 'the'),
                (NOUN, 'sandwich'),
                (VERB, 'was'),
                (ADJ , 'tasty')],
               [(NOUN, 'it'),
                (VERB, 'ate'),
                (NOUN, 'it'),
                (ADJ , 'all')] ]




In [95]:
print('An example from the dataset')
print(my_dataset[0])

An example from the dataset
[(1, 'the'), (2, 'monster'), (3, 'ate'), (1, 'a'), (4, 'big'), (2, 'sandwich')]


In [0]:
vw = pyvw.vw("--search 5 --audit --quiet --search_task hook --ring_size 1024")

# search space of 5 labels

In the below class,
the tweak that reduces the search space by 
introducing a if else statement has been put within 
###################

    _tweak_

###################

In [0]:
count = 0
class SequenceLabeler(pyvw.SearchTask):
    def __init__(self, vw, sch, num_actions):
        pyvw.SearchTask.__init__(self, vw, sch, num_actions)

    def _run(self, sentence):
        output = []
        loss = 0.
        print('\nthe sentence is: ',sentence)

        print('the predictions are:')
        for n in range(len(sentence)):
            pos, word = sentence[n]
            prevPred = output[n-1] if n > 0 else '<start>'
            print(prevPred, type(prevPred))



            ex = self.vw.example({'w': [word], 'p': [prevPred]})
            
            pred = self.sch.predict(examples=ex, my_tag=n+1, oracle=pos, condition=(n,'p'))
            vw.finish_example([ex])

            
            output.append(pred)
            
            #############################################################################################
            
            # the tweak or manual if else conditions to be given here

            
            if len(output) >= 1:
              
              if pred == 5:
                # custom rule can be given
                # as of now just checking for the dummy pos tag.

                print('found a rule match')
                print('-------- prediction is :', pred)
                print('need to break search phase here')
                print('*'*40, '<<<<<')
                output = []
                # no loss is charged as manual intervention is done.
                # an empty output list is passed, as this pass is not to be learnt.
                return output
            
            #############################################################################################

            if pred != pos:
                loss += 1.


        print(output)
        self.sch.loss(loss)
        global count
        count+=1
        print('Number of iterations: ',count)
        
        return output
  

The below snippet is where the dataset is passed for learning via searching the space.

**So let's take the first example from the dataset.**

_The monster ate a big sandwhich_

<br>

6 words are present and 5 POS tags to choose from.

So the search space should contain [6*5 + 2] = 32 iterations.

<br>

However ideally we should be going through [6*4 + 2] = 26 

iterations only as 5th POS tag is dummy and needs to be ruled out from the search space.

<br>

If we remove the if condition, 32 iterations are made for 1st 

example, if we keep the if condition, 26 iterations are made,

reducing our search space.




In [101]:
sequenceLabeler = vw.init_search_task(SequenceLabeler)
sequenceLabeler.learn(my_dataset)



the sentence is:  [(1, 'the'), (2, 'monster'), (3, 'ate'), (1, 'a'), (4, 'big'), (2, 'sandwich')]
the predictions are:
<start> <class 'str'>
found a rule match
-------- prediction is : 5
need to break search phase here
**************************************** <<<<<

the sentence is:  [(1, 'the'), (2, 'monster'), (3, 'ate'), (1, 'a'), (4, 'big'), (2, 'sandwich')]
the predictions are:
<start> <class 'str'>
1 <class 'int'>
2 <class 'int'>
3 <class 'int'>
1 <class 'int'>
4 <class 'int'>
[1, 2, 3, 1, 4, 2]
Number of iterations:  1

the sentence is:  [(1, 'the'), (2, 'monster'), (3, 'ate'), (1, 'a'), (4, 'big'), (2, 'sandwich')]
the predictions are:
<start> <class 'str'>
1 <class 'int'>
2 <class 'int'>
1 <class 'int'>
1 <class 'int'>
4 <class 'int'>
[1, 2, 1, 1, 4, 2]
Number of iterations:  2

the sentence is:  [(1, 'the'), (2, 'monster'), (3, 'ate'), (1, 'a'), (4, 'big'), (2, 'sandwich')]
the predictions are:
<start> <class 'str'>
1 <class 'int'>
2 <class 'int'>
2 <class 'int'>
1 <class 'i

A test sentence is given to check how it performs

In [0]:
print('\n\nThe prediction for test sentence is')
print(sequenceLabeler.predict( [(1,w) for w in "the sandwich ate a monster".split()] ))